In [1]:
# !/usr/bin/env python

# this is a pyspark file, not python
# functions in this file are meant to read the entire telematic data, and given one driver
# produce a new RDD with 400 trips: 200 from original driver, labeled as '1' and sampled
# from other drivers labeled as '0'

# note: resulting RDD will have 400 trips, the rows of the RDD will be individual observations
# from each of those trips - therefore, there will be more then 400 rows

# note: these functions expect the data to be stored as multiple CSV files, one for each driver,
# stored in the same directory, with the format: 'x, y, driverID, tripID, step'
# and named as '1.csv', '2.csv', etc.

# note: these functions will output in the format: 'x, y, driverID, tripID, step, label'

# import modules
import numpy as np
import os as os
import re as re
import math
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

__author__ = "Su-Young Hong AKA Da Masta Killa AKA Synth Pop Rocks of Locks AKA Intergalactic Chilympian"
__status__ = "Prototype"

# read directory of files and return a list of all driverIDs from csv's insid directory
def get_drivers(dirpath):
    """
    :param dirpath: string, path to directory containing driver csv's
    :return: list, contains all driverIDs as strings
    """
    try:
        allfiles = os.listdir(dirpath)
        drivers = [re.sub(r'[^0-9]', '', i) for i in allfiles]
        drivers.remove('')
        return drivers
    except Exception as e:
        print e

# produces random samples of driverIDs and tripIDs in two separate lists
def random_samples(targ_driv, driv_list, K=200):
    """
    :param targ_driv: str, driverID we want to make false trips for
    :param driv_list: list, list of all drivers, produced by get_drivers()
    :param K: number of trips we want to make for targ_driv
    :return: tuple of lists, first list is random driverIDs, second list is list of tripIDs, both are strings
    """
    try:
        driv_list.remove(targ_driv) #removes the target driver from list of drivers to sample from
        drivers = np.random.choice(driv_list, K, True)
        trips = np.random.choice(np.arange(1,K+1).astype(str), K, True)
        return (drivers, trips)
    except Exception as e:
        print e

# reads directory of files and returns RDD of observations from trips in the sample (driverID, tripID combo)
# NOTE: this function is VERY SLOW, it is what slows the entire workflow down
def sample_data(path, driverIDs, tripIDs):
    """
    :param path: string, path to directory containing driver.csv's
    :param driverIDs: list, list of randomly sampled driverIDs as strings, produced by random_sample()
    :param tripIDs: list, list of randomly sampled tripIDs as strings, produced by random_samples()
        NOTE: the above two zip into a list of (driverID, tripID) tuples, with each tuple being a single item in the
        sample
    :return: RDD, contains only observations from the sample
    """
    try:
        combos = zip(driverIDs, tripIDs)
        samplefiles = [path + '/' + 'driver_' + i + '.csv' for i in driverIDs]
        samplefiles = ','.join(set(samplefiles))  #### NOTE: this set() action is a hack for small num. files
        RDD = sc.textFile(samplefiles)   #### NOTE: with large num. files, might need to set num. partitions
        RDDsplit = RDD.map(lambda x: x.split(','))
        RDDsamples = RDDsplit.filter(lambda x: (x[2],x[3]) in combos)
        RDDsamples.cache()
        return RDDsamples
    except Exception as e:
        print e

# takes RDD of samples and assigns new driverID and tripID to observations in a new RDD
def ID_Data(targ_driver, RDD, K = 200):
    """
    :param targ_driver: string, target driver we used to generate samples
    :param RDD: RDD, trip data RDD produced by sample_data(), format will be original form (x,y,driverID,tripID,step)
    :param K: int, number of trips we sampled
    :return: RDD, in original format, but with driverID and tripID changed to look like new observations of the target
    driver
    """
    try:
        newID1 = [targ_driver] * K
        newID2 = np.arange(200, 201+K).astype(str)
        newID = zip(newID1, newID2)
        oldID = RDD.map(lambda x: (x[2],x[3])).distinct().collect()
        glossary = sc.parallelize(zip(oldID, newID))
        newRDD = RDD.map(lambda x: ((x[2],x[3]), ([x[0],x[1],x[4]]))).join(glossary)
        newID_RDD = newRDD.map(lambda x: (x[1][0][0], x[1][0][1], x[1][1][0], x[1][1][1], x[1][0][2]))
        return newID_RDD
    except Exception as e:
        print e


# takes RDD in original form and converts it into key-value tuple with values being x,y,step,label
def processRDD(RDD, label):
    """
    :param RDD: RDD in original format (x,y,driverID,tripID,step)
    :param label: category of observation, 1 for positive, 0 for negative
    # note, not sure if it needs to be int or float
    :return: RDD, RDD returned in new key/value format: (driverID, tripID), (x, y, step, label)
    # note, x, y, step, and label will be floats
    """
    try:
        newRDD = RDD.map(lambda x: ((x[2],x[3]),(float(x[0]),float(x[1]),float(x[4]),label)))
        return newRDD
    except Exception as e:
        print e

# takes a driver to target, path to directory of driver.csv's, and returns an RDD labeled with
# (driverID, tripID),(x,y,step,label), where a label 1 is from an actual trip, and label 0 is from
# a trip randomly sampled from other drivers
def labelRDDs(targ_driv, path, K=200):
    """
    :param targ_driv: string, driver we want to create positive and negative labeled data for
    :param path: string, path to directory where driver.csvs are stored
    :param K: int, number of negative (manufactured) trips to sample
    :return: RDD with key, value tuple where key is (driverID, tripID) and value is (x,y,step,label)
    """
    try:
        target = sc.textFile(path + '/' + 'driver_' + targ_driv + '.csv') #load target driver's data
        target2 = target.map(lambda x: x.split(',')) #convert from string to list of strings
        positives = processRDD(target2, 1.0) #label target driver's RDD
        driv_lis = get_drivers(path) #get python list of all possible drivers to sample from
        sampdriv, samptrip = random_samples(targ_driv, driv_lis, K) #generate random samples of drivers and tripIDs
        samples = sample_data(path, sampdriv, samptrip) #generate RDD of random samples
        samplesRDD = ID_Data(targ_driv, samples, K) #relabel samples to look like target driver's trips
        negatives = processRDD(samplesRDD, 0.0) #label samples
        finalRDD = positives.union(negatives).cache() #join target driver and samples together
        return finalRDD
    except Exception as e:
        print e



In [9]:
def vectorRDD(RDD):
    """
    :param RDD: RDD, created from labelRDDs
    :return: RDD with driver_id, trip_id, vectorized x and y coordinates, 
    step number, and label 
    """

    # The RDD created from labelRDDs is first mapped into the key, value pair
    # ((driver_id, trip_id), (x, y, step)). Each element in the value is a
    # list of one element. This is so we can create a list of each x, y, and 
    # step coordinate when reducing. We then reduce by key creating a list of
    # x and y coordinates, and a list of trip step number. This will create an
    # RDD of the form ((driver_id, trip_id), ([x coordinates], 
    # [y coordinates], [trip steps])). Finally, we map this into the key, value 
    # pair ((driver_id, trip_id), ([x cooridinates], [y coordinates], 
    # [trip steps], label)). This RDD is the returned.

    vectorRDD = RDD.map(lambda x: (x[0], ([x[1][0]], [x[1][1]], 
        [x[1][2]])))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2]))\
    .map(lambda x: (x[0], (x[1][0], x[1][1], x[1][2], 1)) if int(x[0][1])\
        < 201 else (x[0], (x[1][0], x[1][1], x[1][2], 0)))
    return vectorRDD


def get_polars(RDD):
    """
    :param RDD: RDD, created from vectorRDD
    "return: RDD, same as vectorRDD but also with polar coordinates
    """

    # The RDD created from vectorRDD is mapped into the key, value pair
    # ((driver_id, trip_id), ([x coordinates], [y coordinates], 
    # [r coordinates], [theta coordinates], [step numbers], label)). Here,
    # the r coordinate is the radial coordinate from the origin, computed using
    # Euclidean distance, i.e. the first inner map statement. The theta
    # coordinate is the angular polar coordinate, computed as the arctan of 
    #  y / x, i.e. the second inner map statement.

    polars = RDD.map(lambda x: (x[0], (x[1][0], x[1][1], 
        map(lambda x: (x[0] ** 2 + x[1] ** 2) ** 0.5, zip(x[1][0], x[1][1])), 
        map(lambda x: math.atan2(x[1], x[0]), zip(x[1][0], x[1][1])),
         x[1][2], x[1][3])))
    return polars


def step_level_features(polarRDD):
    """
    :param RDD: RDD, created from get_polars
    :return: RDD with speed and acceleration at each stage of the trip
    """

    # First, the RDD created from get_polars is mapped to the key, value pair
    # ((driver_id, trip_id), ([x coordinates], [y coordinates],
    # [r coordinates], [theta coordinates], [v coordinates], [step numbers], 
    # label)). v is the current speed (m/s), which is computed from the 
    # euclidean, distance between the current point and the previous point, 
    # i.e. the two innermost maps square the individual coordinate differences.
    # These are then added and square rooted for speed. The second step maps
    # this key, value pair into the key, value pair ((driver_id, trip_id), 
    # ([x coordinates], [y coordinates], [r coordinates], [theta coordinates], 
    # [v coordinates], [a coordinates], label)). a is the current acceleration,
    # which is computed as the difference between the current speed and the 
    # previous speed

    step_lv = polarRDD.map(lambda x: (x[0], (x[1][0], x[1][1], x[1][2], 
        x[1][3], map(lambda x: (x[0] + x[1]) ** 0.5, 
            zip(map(lambda x: (x[0] - x[1]) ** 2, 
                zip(x[1][0], [0.0] + x[1][0][:-1])), 
            map(lambda x: (x[0] - x[1]) ** 2, 
                zip(x[1][1], [0.0] + x[1][1][:-1])))), x[1][4], x[1][5])))\
    .map(lambda x: (x[0], (x[1][0], x[1][1], x[1][2], x[1][3], x[1][4], 
        map(lambda x: x[0] - x[1], 
            zip(x[1][4], [0.0] + x[1][4][:-1])), x[1][5], x[1][6])))

    return step_lv


def trip_level_features(RDD):  
    """
    :param RDD: RDD, created from step_level_features
    :return: RDD with features, aggregated over the trip
    """ 

    # The RDD created from step_level_features is mapped to the key, value
    # pair ((driver_id, trip_id), (min(v), max(v), min(a), max(a), trip
    # length, trip distance, mean(v), stddev(v), mean(a), stddev(a), 
    # length of time stopped, label). Trip length (s) is the number of 
    # coordinates. Trip distance (m) is computed as the sum of the speeds as 
    # they are computed per second. Length of time stopped is computed as the
    # number of seconds where the speed is less than 0.5 m/s.

    trip_lv = RDD.map(lambda x: (x[0], (min(x[1][4]), max(x[1][4]), 
        min(x[1][5]), max(x[1][5]), len(x[1][0]), sum(x[1][4]), 
        np.mean(x[1][4]), np.std(x[1][4]), np.mean(x[1][5]), np.std(x[1][5]), 
        sum([elem < 0.5 for elem in x[1][4]]), x[1][7])))

    return trip_lv

In [10]:

path = '/Users/coolguy/Desktop/ML2Project/Data/processed_data/sample_drivers'
driver = '1'
example1 = labelRDDs(driver, path)


In [11]:
example1.count()

233566

In [12]:
example2 = labelRDDs('2', path)

In [13]:
example2.count()

272575

In [14]:
example1 = vectorRDD(example1)
example2 = vectorRDD(example2)

In [15]:
example1 = get_polars(example1)
example2 = get_polars(example2)

In [16]:
example1.first()[1][5]

0

In [17]:
example1 = step_level_features(example1)
example2 = step_level_features(example2)
#index 5 and 6 are speed and accel

In [18]:
example1.first()[1][5]

[0.0,
 0.4242640687119285,
 0.4242640687119286,
 0.2144864438496077,
 0.7397610564585297,
 0.5741972270689477,
 0.17253689199545041,
 0.273609085902228,
 0.04044536995664938,
 0.03815941305411208,
 -0.4319058186636875,
 -0.24945747592824397,
 0.5455030060704162,
 0.5356514666559686,
 -0.23394247380824185,
 0.07089863525944873,
 0.5736712733590759,
 -1.0872612889727815,
 0.7809963235039419,
 -0.9058772731852764,
 -0.8721179403900323,
 -0.7059276138806819,
 0.244235933239772,
 -1.16619037896906,
 0.0,
 1.063014581273465,
 -1.063014581273465,
 0.14142135623730775,
 -0.14142135623730775,
 0.0,
 0.0,
 0.14142135623731025,
 0.424264068711927,
 1.2453916026782466,
 0.9475458171992606,
 0.5715423162425979,
 0.43546891568320323,
 0.1394907612007823,
 1.110849643640452,
 0.019896843211888893,
 -0.9542049984139593,
 1.0806974732644141,
 -0.2797415533626424,
 0.41832106598594887,
 0.642961128406002,
 -1.1272666091685162,
 -0.6573109628998308,
 0.36836570045162453,
 -1.1218152958834309,
 0.01464900

In [19]:
A = [1,2,3,4]

In [20]:

def bin_accel_events(accel_list):
    bins = []
    for i in range(len(accel_list)):
        if i == 0:
            bins.append([accel_list[i]])
        else:
            if cmp(accel_list[i],0) == cmp(accel_list[i-1],0):
                bins[-1] += [accel_list[i]]
            else: bins.append([accel_list[i]])
    return bins


In [21]:
hmm = bin_accel_events(example1.first()[1][5])

In [66]:
np.mean(example1.first()[1][5])

2.6161367296027252e-18

In [51]:
def bin_accel_events_with_index(accel_list):
    bins = []
    temp = []
    for i in range(len(accel_list)):
        if i == 0:
            temp.append(accel_list[i])
        else:
            if cmp(accel_list[i],0) == cmp(accel_list[i-1],0):
                temp.append(accel_list[i])
            else: 
                bins.append((temp, i-1))
                temp = [accel_list[i]]
        if i == len(accel_list) - 1:
            bins.append((temp, i))
    return bins


In [52]:
rmmm = bin_accel_events_with_index(example1.first()[1][5])

In [53]:
rmmm

[([0.0], 0),
 ([0.4242640687119285,
   0.4242640687119286,
   0.2144864438496077,
   0.7397610564585297,
   0.5741972270689477,
   0.17253689199545041,
   0.273609085902228,
   0.04044536995664938,
   0.03815941305411208],
  9),
 ([-0.4319058186636875, -0.24945747592824397], 11),
 ([0.5455030060704162, 0.5356514666559686], 13),
 ([-0.23394247380824185], 14),
 ([0.07089863525944873, 0.5736712733590759], 16),
 ([-1.0872612889727815], 17),
 ([0.7809963235039419], 18),
 ([-0.9058772731852764, -0.8721179403900323, -0.7059276138806819], 21),
 ([0.244235933239772], 22),
 ([-1.16619037896906], 23),
 ([0.0], 24),
 ([1.063014581273465], 25),
 ([-1.063014581273465], 26),
 ([0.14142135623730775], 27),
 ([-0.14142135623730775], 28),
 ([0.0, 0.0], 30),
 ([0.14142135623731025,
   0.424264068711927,
   1.2453916026782466,
   0.9475458171992606,
   0.5715423162425979,
   0.43546891568320323,
   0.1394907612007823,
   1.110849643640452,
   0.019896843211888893],
  39),
 ([-0.9542049984139593], 40),
 ([1

In [42]:
[i[0][0] for i in rmmm]

[[0.0],
 0.4242640687119285,
 -0.4319058186636875,
 0.5455030060704162,
 -0.23394247380824185,
 0.07089863525944873,
 -1.0872612889727815,
 0.7809963235039419,
 -0.9058772731852764,
 0.244235933239772,
 -1.16619037896906,
 0.0,
 1.063014581273465,
 -1.063014581273465,
 0.14142135623730775,
 -0.14142135623730775,
 0.0,
 0.14142135623731025,
 -0.9542049984139593,
 1.0806974732644141,
 -0.2797415533626424,
 0.41832106598594887,
 -1.1272666091685162,
 0.36836570045162453,
 -1.1218152958834309,
 0.014649002112138376,
 -0.5879443701842524,
 0.00496444630979731,
 -0.05468243919476379,
 0.12107969809977437,
 -0.23625480431713974,
 0.5772600688626133,
 -0.09328769636617107,
 0.08320444365405955,
 -0.2931792155598103,
 1.722402332269624,
 -1.2048344259758927,
 0.12380880291730989,
 -0.1619070703083949,
 0.12533098598656878,
 -0.19062979403668834,
 0.03067503400318472,
 -0.15854327248065303,
 0.029128236954765185,
 -1.1296613986161503,
 1.2288827674396305,
 -0.7788333004059336,
 0.351983966344342

In [54]:
def filter_accel(accel_bins_with_index):
    filtered = [i for i in accel_bins_with_index if i[0][0] > 0]
    return filtered

# takes input of above and returns just events that are negative
def filter_deccel(accel_bins_with_index):
    filtered = [i for i in accel_bins_with_index if i[0][0] < 0]
    return filtered

In [61]:
decelEvents = filter_deccel(rmmm)
accelEvents = filter_accel(rmmm)

In [48]:
[i[0] for i in decelEvents]

[[[0.0]],
 [0.4242640687119285,
  0.4242640687119286,
  0.2144864438496077,
  0.7397610564585297,
  0.5741972270689477,
  0.17253689199545041,
  0.273609085902228,
  0.04044536995664938,
  0.03815941305411208],
 [0.5455030060704162, 0.5356514666559686],
 [0.07089863525944873, 0.5736712733590759],
 [0.7809963235039419],
 [0.244235933239772],
 [1.063014581273465],
 [0.14142135623730775],
 [0.14142135623731025,
  0.424264068711927,
  1.2453916026782466,
  0.9475458171992606,
  0.5715423162425979,
  0.43546891568320323,
  0.1394907612007823,
  1.110849643640452,
  0.019896843211888893],
 [1.0806974732644141],
 [0.41832106598594887, 0.642961128406002],
 [0.36836570045162453],
 [0.014649002112138376, 1.1935825227263126],
 [0.00496444630979731,
  0.03949761326024959,
  0.4944583967958076,
  0.3927372182908817,
  0.18999296799728782],
 [0.12107969809977437,
  0.6719073743396695,
  0.7844583038949624,
  0.7949735493372909],
 [0.5772600688626133],
 [0.08320444365405955, 1.2990902246272524, 0.163

In [57]:
def mean_accel_events(bined_accels_with_index):
    events = [i[0] for i in bined_accels_with_index]
    means = [sum(i)/float(len(i)) for i in events]
    overall_mean = sum(means)/float(len(means))
    return overall_mean

In [62]:
meenpos = mean_accel_events(accelEvents)
meenneg = mean_accel_events(decelEvents)

In [68]:
meenpos

0.58021125093269

In [56]:
[sum(j)/float(len(j)) for j in [i[0] for i in decelEvents]]

[0.32241373618993135,
 0.5405772363631924,
 0.3222849543092623,
 0.7809963235039419,
 0.244235933239772,
 1.063014581273465,
 0.14142135623730775,
 0.559541258311741,
 1.0806974732644141,
 0.5306410971959754,
 0.36836570045162453,
 0.6041157624192255,
 0.2243301285308048,
 0.5931047314179243,
 0.5772600688626133,
 0.5152248508657497,
 1.2219829070488892,
 0.8290712894535165,
 0.12533098598656878,
 0.03067503400318472,
 0.029128236954765185,
 1.2288827674396305,
 0.35198396634434204,
 1.1221359582841242,
 0.9615628797034326,
 1.1528633436329478,
 1.1435725536146784,
 0.7179724597216577,
 0.4246469220066859,
 0.15636606055316093,
 0.31427467192718694,
 0.3413562951050899,
 0.4465391256554412,
 0.5176423021232708,
 0.6297827522015975,
 0.55207972893962,
 0.06180339887500354,
 8.038014698286133e-14,
 1.063014581273466,
 0.22360679774995357,
 0.07071067811867082,
 0.14142135623730148,
 1.1633738382616192,
 0.8324045243385397,
 0.8725009202016958,
 0.6919193080669785,
 1.32328371814895,
 0.8

In [ ]:
sum([0.12360679774998887])

In [69]:
decelEvents

[([-0.4319058186636875, -0.24945747592824397], 11),
 ([-0.23394247380824185], 14),
 ([-1.0872612889727815], 17),
 ([-0.9058772731852764, -0.8721179403900323, -0.7059276138806819], 21),
 ([-1.16619037896906], 23),
 ([-1.063014581273465], 26),
 ([-0.14142135623730775], 28),
 ([-0.9542049984139593], 40),
 ([-0.2797415533626424], 42),
 ([-1.1272666091685162, -0.6573109628998308], 46),
 ([-1.1218152958834309], 48),
 ([-0.5879443701842524], 51),
 ([-0.05468243919476379,
   -0.0811469968420715,
   -0.011985634456666006,
   -0.2703065638458133],
  60),
 ([-0.23625480431713974], 65),
 ([-0.09328769636617107], 67),
 ([-0.2931792155598103], 71),
 ([-1.2048344259758927], 77),
 ([-0.1619070703083949], 83),
 ([-0.19062979403668834], 85),
 ([-0.15854327248065303, -0.9092771675843316, -0.09577416563068653], 89),
 ([-1.1296613986161503, -0.874282148590023], 92),
 ([-0.7788333004059336], 94),
 ([-0.11685479659440468], 96),
 ([-1.770693837081966], 98),
 ([-0.25745231370072297], 100),
 ([-0.94249304460457

In [76]:
[len(j) for j in [i[0] for i in decelEvents]]

[2,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 4,
 4,
 1,
 1,
 9,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 18,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 10,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 1,
 2,
 4,
 1,
 3,
 1,
 13,
 2,
 2,
 1,
 4,
 1,
 3,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 3,
 1,
 4,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1]

In [77]:
[len(j) for j in [i[0] for i in accelEvents]]

[9,
 2,
 2,
 1,
 1,
 1,
 1,
 9,
 1,
 2,
 1,
 2,
 5,
 4,
 1,
 3,
 5,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 9,
 7,
 2,
 1,
 1,
 2,
 7,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 6,
 4,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 3,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 15,
 1,
 1,
 4,
 4,
 2,
 1,
 1,
 2,
 1,
 2,
 3,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 9,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1]